In [ ]:
import pandas as pd
import logging
import numpy as np
from icecream import ic
from itertools import product
from geopy.distance import geodesic
from dataclasses import dataclass
import functools


logging.basicConfig(level=logging.DEBUG)

In [ ]:
PATH = "./cities/"
INSTANCE = "italy.csv"

In [ ]:
cities = pd.read_csv(f"{PATH}{INSTANCE}", header=None, names=["City", "lat", "long"])
#cities

In [ ]:
#show only first column
cities_names = np.array([c['City'] for _,c in cities.iterrows()])
#cities_names

In [ ]:
coordinates = cities[["lat","long"]].to_numpy()
#coordinates

## Helper funciton

In [ ]:
def distance(c1,c2):
    return geodesic(c1,c2).km

#distance(coordinates[0],coordinates[1])

In [ ]:
dist_matrix = np.array([[distance(c1,c2) for c1 in coordinates] for c2 in coordinates])

In [ ]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


@counter
def tsp_cost(tsp):
#    ic(tsp[0], tsp[-1])
#    assert tsp[0] == tsp[-1]
#    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost

In [ ]:
def greedy_tsp():
    visited = [False]*len(coordinates)
    dist = dist_matrix.copy()

    city = 0
    tsp = list()
    #ic(dist)
    tsp.append(city)

    while not np.all(visited):
        min_dist = np.inf
        next_city = None
        for i in range(len(coordinates)):
            if not visited[i] and dist[city][i] < min_dist and i != city:
                next_city = i
                min_dist = dist[city][i]
        visited[next_city] = True
        tsp.append(next_city)
        #ic("visiting city: ", cities_names[city])
       # logging.debug(
           # f"step: {cities_names[city]} -> {cities_names[next_city]} ({min_dist:.2f}km)")
        city = next_city
        
                

    tot_cost = 0
   # for c1, c2 in zip(tsp, tsp[1:]):
    #    tot_cost += dist_matrix[c1, c2]
    #logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")  
    return tsp

## Greedy with random starting point

In [ ]:
def greedy_tsp_random_start(start):
    visited = [False]*len(coordinates)
    dist = dist_matrix.copy()

    city = start
    visited[city] = True

    tsp = list()
    #ic(dist)
    tsp.append(city)

    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        # logging.debug(
        #     f"step: {cities_names[city]} -> {cities_names[closest]} ({dist[city,closest]:.2f}km)")
       
        visited[closest] = True
        city = closest
        tsp.append(int(city))
  
    # logging.debug(
    #     f"step: {cities_names[tsp[-1]]} -> {cities_names[tsp[0]]} ({dist[tsp[0],tsp[-1]]:.2f}km)")
    tsp.append(tsp[0])
    
            
    tot_cost =tsp_cost(tsp)
    #logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")  
    return tsp

In [ ]:
""" best_sol = -1
best_cost = np.inf

for i in range(len(cities)):
    sol = greedy_tsp_random_start(i)
    cost = tsp_cost(sol)
    # logging.info(f"initial_sol: {sol}")
    # logging.info(f"initial_cost: {cost}")
    if cost < best_cost:
        best_sol = sol
        best_cost = cost

logging.info(f"best_sol: {best_sol}")
logging.info(f"best_cost: {best_cost}") """

## gready + EA

### helper functions

In [ ]:
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

def fitness(individual):
    return  -float(tsp_cost(individual.genome))

def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


In [ ]:
def pmover(p1, p2):
    i1 = np.random.randint(len(p1))
    i2 = np.random.randint(len(p1))
        # i1 = 3
    # i2 = 6
    while i1 == i2:
        i2 = np.random.randint(len(p1))
  #  ic(i1, i2)

    if i1 > i2:
        i1, i2 = i2, i1
 

    o1 = p1.copy()
    o2 = p2.copy()
    o1[i1:i2] = p2[i1:i2]
    o2[i1:i2] = p1[i1:i2]

    for i in range(0, i1):
        while o1[i] in o1[i1:i2]:
            o1[i] = p1[p2.index(o1[i])]
        while o2[i] in o2[i1:i2]:
            o2[i] = p2[p1.index(o2[i])]

    for i in range(i2, len(o1)):
        while o1[i] in o1[i1:i2]:
            o1[i] = p1[p2.index(o1[i])]
        while o2[i] in o2[i1:i2]:
            o2[i] = p2[p1.index(o2[i])]
    
   # ic(p1,p2)
   # ic(o1,o2)
    return o1, o2

# p1 = [1, 2, 3, 4, 5, 6 ,7, 8]
# p2 = [3, 7, 5, 1, 6, 8, 2, 4]
# # p1 = np.ndarray(p1)
# # p2 = np.ndarray(p2)
# o1,o2 = pmover(p1, p2)
# ic(o1)
# ic(o2)

In [297]:
def inv_mutation(individual):
    mutated_genome = individual.genome.copy()
 
     # Select two random indices for the inversion segment
    i, j = sorted(np.random.randint(0, len(individual.genome), 2))
    while i == j:
        i, j = sorted(np.random.randint(0, len(individual.genome), 2))
   # ic(i, j)
    mutated_genome[i:j+1] = mutated_genome[i:j+1][::-1]

    return mutated_genome
    


In [ ]:
POPULATION_SIZE = 100
population = [Individual(greedy_tsp_random_start(np.random.randint(len(cities)))) for _ in range(POPULATION_SIZE)]
population.sort(key=lambda i: tsp_cost(i.genome))
ic(tsp_cost(population[0].genome))


for i in population:
    #remove last one
    i.genome = i.genome[:-1]
    i.fitness = -tsp_cost(i.genome)
    

OFFSPRING_SIZE = 10
MAX_GENERATIONS = 10_000

for g in range(MAX_GENERATIONS):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # HYPERMODERN
        if np.random.random() < 0.3:
            p = parent_selection(population)
            o = inv_mutation(p)
            o = Individual(o)
            offspring.append(o)
            
        else :
            p1 = parent_selection(population)
            p2 = parent_selection(population)
            o1,o2 = pmover(p1.genome, p2.genome)
            o1,o2 = Individual(o1), Individual(o2)
        
            offspring.append(o1)
            offspring.append(o2)

    for i in offspring:
        i.fitness = fitness(i)
        # ic(i.genome)
        # ic(i.fitness)

    population.extend(offspring)
#    ic(-population[0].fitness)
 #   ic(-population[-1].fitness)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
  #  ic(-population[0].fitness)
   # ic(-population[-1].fitness)


for i in population:
    #add last one
    i.genome = np.append(i.genome, i.genome[0])
    i.fitness = -tsp_cost(i.genome)

population.sort(key=lambda i: i.fitness, reverse=True)
ic(-population[0].fitness, tsp_cost.calls)

ic| tsp_cost(population[0].genome): np.float64(4436.03176952516)
ic| -population[0].fitness: np.float64(4350.277436006993)
    tsp_cost.calls: 2304332


(np.float64(4350.277436006993), 2304332)